# Load data from backend

In [1]:
import sys
import requests
import json

headers = {'Cookie': '_orca_session=vfpsRAbdaZa%2FFTPluXKpRiqvK1W%2F19fuTbC3TEwkfk0k1FFXrFQ1WVIUI8%2BmBIlIL3m%2FvOd7CAbLiDNaBTBzvAmx%2F%2FFqOMnBkTPgHj4cU9Qp1ImdN7mf10uaKoW7hiw3phGCEn9kacKNrbHXjE7Jq%2BzK%2BOERk2PiCwhIHZJKPMAPz%2BOBMnXIb267YXOEzgpWUriBcqB4CfsHL1rUJtbLEUuKVz2%2BuREHVzCSxMf%2B7AXfxTscYcraQA4ICJvcUuhwJlkhk1BWsmp4cUiJUh67c%2BIDH7KiKlhPQh4aZ7vc0Avg8JQJVfWYRENVfuTh8sJ%2F%2F3EfknbuN37IUthOm8AU2kbddDz0--IiufLnvRxR%2FEXciC--OS%2FQByuDwJHsv3pp0lyyXw%3D%3D'}
UNIT_URL = "https://orca.mova21.ch/units/"
ACTIVTY_URL = "https://orca.mova21.ch/activities/"
ACTIVTY_URL_JSON = "https://orca.mova21.ch/activities.json"

# get activities from orca
r = requests.get(ACTIVTY_URL_JSON, headers=headers)
activities = r.json()

# read all units from backend
units = []
for unit_id in range(1, 1010 + 1):
  print("Reading unit " + str(unit_id))
  r = requests.get(UNIT_URL + str(unit_id) + ".json", headers=headers)
  units.append(r.json())

Reading unit 1
Reading unit 2
Reading unit 3
Reading unit 4
Reading unit 5
Reading unit 6
Reading unit 7
Reading unit 8
Reading unit 9
Reading unit 10
Reading unit 11
Reading unit 12
Reading unit 13
Reading unit 14
Reading unit 15
Reading unit 16
Reading unit 17
Reading unit 18
Reading unit 19
Reading unit 20
Reading unit 21
Reading unit 22
Reading unit 23
Reading unit 24
Reading unit 25
Reading unit 26
Reading unit 27
Reading unit 28
Reading unit 29
Reading unit 30
Reading unit 31
Reading unit 32
Reading unit 33
Reading unit 34
Reading unit 35
Reading unit 36
Reading unit 37
Reading unit 38
Reading unit 39
Reading unit 40
Reading unit 41
Reading unit 42
Reading unit 43
Reading unit 44
Reading unit 45
Reading unit 46
Reading unit 47
Reading unit 48
Reading unit 49
Reading unit 50
Reading unit 51
Reading unit 52
Reading unit 53
Reading unit 54
Reading unit 55
Reading unit 56
Reading unit 57
Reading unit 58
Reading unit 59
Reading unit 60
Reading unit 61
Reading unit 62
Reading unit 63
R

# Class Definitions

In [2]:
# super class of activity
class Activity:
  def __init__(self, id, label, category, activity_executions):
    self.id = id
    self.label = label
    self.category = category
    self.activity_executions = activity_executions
    self.weighted_number_of_bookings = 0
    self.booked_units = []
  
  def __repr__(self):
    return "id: " + str(self.id) + "\n label: " + self.label

  def addUnit(self, unit):
    self.booked_units.append(unit)

  def getNumberOfBookings(self):
    return len(self.booked_units)

  def getNumberOfPeopleBooked(self):
    return sum(unit.participant_count for unit in self.booked_units)

  def getExecutionCapacity(self):
    return sum(activity_execution.amount_participants for activity_execution in self.activity_executions)

# representing activity object which units have booked, based on activity
# selected activity has further a priority
class SelectedActivity(Activity):
  def __init__(self, id, label, category, priority, activity_executions):
    super().__init__(id, label, category, activity_executions)
    self.priority = priority

  def __repr__(self):
    return str(self.id) + " - " + str(self.priority)

  # returns a score based on the priority used for the weighted summarization
  def getWeightedPriorityScore(self):
    # +1 to ensure none zero terms
    return (1/(self.priority + 1))**(1/3)

In [3]:
class ActivityExecutions:
  def __init__(self, id, amount_participants, starts_at, ends_at, languages, spot, transport):
    self.id = id
    self.amount_participants = amount_participants
    self.starts_at = starts_at
    self.ends_at = ends_at
    self.languages = languages
    self.spot = spot
    self.transport = transport

In [4]:
class Unit:
  def __init__(self, id, district, starts_at, ends_at, language, participant_count, leader_count, stufe, week, selected_activities):
    self.id = id
    self.district = district
    self.starts_at = starts_at
    self.ends_at = ends_at
    self.language = language
    self.participant_count = participant_count
    self.leader_count = leader_count
    self.stufe = stufe
    self.week = week
    self.selected_activities = selected_activities       # activities booked by the unit

  def __repr__(self):
    return "id" + str(self.id)

In [5]:
# class which combines information and can be used to generate reports
class Report:
  def __init__(self):
    self.acitivites = []
    self.units = []
  
  def addActivity(self, activity):
    self.acitivites.append(activity)

  def addUnit(self, unit):
    self.units.append(unit)

    # add unit to report and to the corresponding activity object
    for sel_act in unit.selected_activities:
      act = self.getActivity(sel_act.id)
      
      if act:
        act.addUnit(unit)
        act.weighted_number_of_bookings += sel_act.getWeightedPriorityScore()
      else:
        raise Exception('Unit booked an none existing activity :(')

  # get activity object based on id
  def getActivity(self, id):
    return next((activity for activity in self.acitivites if activity.id == id), None)
  
  # generate a csv export of all bookings, each unit is listed for each of their bookings
  def getListOfBookings(self):
    logger.write("unit_id;" + "unit_stufe;" + "unit_language;" + "unit_#participant;" + "unit_#leaders;" + "unit_url;" + "act_id;" + "prio;" + "activity_url")
    for unit in self.units:
      for sel_act in unit.selected_activities:
        unit_url = UNIT_URL + str(unit.id)
        activity_url = ACTIVTY_URL + str(sel_act.id)
        logger.write(str(unit.id) + ";" + str(unit.stufe) + ";" + str(unit.language) + ";" + str(unit.participant_count) + ";" + str(unit.leader_count) + ";" + unit_url + ";" + str(sel_act.id) + ";" + str(sel_act.priority) + ";" +  activity_url)

  # generate csv export for activities with their corresponding bookings
  def getBookingCountOfActivity(self):
    sorted_activities = sorted(self.acitivites, key=lambda act: act.getNumberOfBookings(), reverse=True)

    logger.write("act_id;" + "#bookings;" + "#numberOfPeopleAssigned;" + "activity_url")
    for act in sorted_activities:
      activity_url = ACTIVTY_URL + str(act.id)
      logger.write(str(act.id) + ";" + str(act.getNumberOfBookings()) + ";" + str(act.getNumberOfPeopleBooked()) + ";" + activity_url)

  # generate csv export for activities with their corresponding bookings WEIGHTED by their priority
  def getBookingCountOfActivityWeighted(self):
    sorted_activities = sorted(self.acitivites, key=lambda act: act.weighted_number_of_bookings, reverse=True)

    logger.write("act_id;" + "#bookings;" + "activity_url")
    for act in sorted_activities:
      activity_url = ACTIVTY_URL + str(act.id)
      logger.write(str(act.id) + ";" + str(act.weighted_number_of_bookings) + ";" + activity_url)

  def getListOfActivities(self):
    logger.write("act_id;" + "execution_capacity;" + "activity_url")

    for act in self.acitivites:
      activity_url = ACTIVTY_URL + str(act.id)
      logger.write(str(act.id) + ";" + str(act.getExecutionCapacity()) + ";" + activity_url)

In [6]:
class Logger:
  def __init__(self, filename):
    self.console = sys.stdout
    self.file = open(filename, 'w')

  def write(self, message):
    self.console.write(message + '\n')
    self.file.write(message + '\n')

  def flush(self):
    self.console.flush()
    self.file.flush()

# Load scheduler with data

In [7]:
report = Report()

# create activities
for activity in activities:
  act_executions = activity.get("activity_executions")

  executions = []
  for act_exe in act_executions:
    executions.append(ActivityExecutions(
                                         id = act_exe.get("id"), 
                                         amount_participants = act_exe.get("amount_participants"), 
                                         starts_at = act_exe.get("starts_at"), 
                                         ends_at = act_exe.get("ends_at"), 
                                         languages = act_exe.get("languages"), 
                                         spot = act_exe.get("spot"), 
                                         transport = act_exe.get("transport")))
    
  act = Activity(activity.get("id"), 
                 activity.get("label_in_database").get("de"),
                 activity.get("activity_category").get("code"),
                 activity_executions = executions)
  
  report.addActivity(act)

# create units
for unit in units:
  # if unit couldn't be found in backend, invalid unit ID queried
  if unit.get("id") is not None:
    selected_activities = []
    sel_activities = unit.get("unit_activities")

    # if unit did not make any booking
    if sel_activities: 
      for sel_act in sel_activities:
        # create SelectedActivity based on booked activity
        act = report.getActivity(sel_act.get("activity").get("id"))

        # if unit booked a none existing acitivity
        if act:
          selected_activities.append(SelectedActivity(act.id,
                                                      act.label,
                                                      act.category,
                                                      sel_act.get("priority"),
                                                      act.activity_executions))
        else:
          raise Exception('Unit ' + str(unit.get("id")) + ' booked an none existing activity :(')

        # reorder priority, so it is easier interpretable
        selected_activities = sorted(selected_activities, key=lambda selected_activity: selected_activity.priority)
    else:
      print('Unit ' + str(unit.get("id")) + ' did not make any bookings :(')

    # create unit
    un = Unit(
            id = unit.get("id"),
            district = unit.get("district"),
            starts_at = unit.get("starts_at"),
            ends_at = unit.get("ends_at"),
            language = unit.get("language"),
            participant_count = unit.get("participant_role_counts").get("participant") if unit.get("participant_role_counts") else 0,
            leader_count = unit.get("participant_role_counts").get("helper") if unit.get("participant_role_counts") else 0,
            stufe = unit.get("stufe"),
            week = unit.get("week"),
            selected_activities = selected_activities
    )

    report.addUnit(un)
  else:
    print('Unit ' + str(unit.get("id")) + ' does not exist :(')

Unit None does not exist :(
Unit None does not exist :(
Unit None does not exist :(
Unit None does not exist :(
Unit None does not exist :(
Unit None does not exist :(
Unit 39 did not make any bookings :(
Unit None does not exist :(
Unit None does not exist :(
Unit None does not exist :(
Unit None does not exist :(
Unit None does not exist :(
Unit None does not exist :(
Unit None does not exist :(
Unit None does not exist :(
Unit None does not exist :(
Unit None does not exist :(
Unit None does not exist :(
Unit None does not exist :(
Unit None does not exist :(
Unit None does not exist :(
Unit None does not exist :(
Unit None does not exist :(
Unit None does not exist :(
Unit None does not exist :(
Unit None does not exist :(
Unit None does not exist :(
Unit None does not exist :(
Unit None does not exist :(
Unit None does not exist :(
Unit None does not exist :(
Unit None does not exist :(
Unit None does not exist :(
Unit None does not exist :(
Unit None does not exist :(
Unit None d

In [8]:
# Report Activity

In [9]:
logger = Logger(filename="activity_capacity.csv")
report.getListOfActivities()
logger.flush()

act_id;execution_capacity;activity_url
144;800;https://orca.mova21.ch/activities/144
33;800;https://orca.mova21.ch/activities/33
152;0;https://orca.mova21.ch/activities/152
205;960;https://orca.mova21.ch/activities/205
187;0;https://orca.mova21.ch/activities/187
10;300;https://orca.mova21.ch/activities/10
18;1600;https://orca.mova21.ch/activities/18
56;0;https://orca.mova21.ch/activities/56
215;0;https://orca.mova21.ch/activities/215
15;800;https://orca.mova21.ch/activities/15
203;640;https://orca.mova21.ch/activities/203
228;0;https://orca.mova21.ch/activities/228
120;0;https://orca.mova21.ch/activities/120
185;0;https://orca.mova21.ch/activities/185
12;276;https://orca.mova21.ch/activities/12
116;0;https://orca.mova21.ch/activities/116
141;160;https://orca.mova21.ch/activities/141
235;0;https://orca.mova21.ch/activities/235
104;0;https://orca.mova21.ch/activities/104
57;0;https://orca.mova21.ch/activities/57
59;0;https://orca.mova21.ch/activities/59
119;0;https://orca.mova21.ch/activ

# Report Units

In [10]:
logger = Logger(filename="bookings.csv")
report.getListOfBookings()
logger.flush()

Streaming output truncated to the last 5000 lines.
651;wolf;de;7;0;https://orca.mova21.ch/units/651;46;2;https://orca.mova21.ch/activities/46
651;wolf;de;7;0;https://orca.mova21.ch/units/651;74;3;https://orca.mova21.ch/activities/74
651;wolf;de;7;0;https://orca.mova21.ch/units/651;151;4;https://orca.mova21.ch/activities/151
651;wolf;de;7;0;https://orca.mova21.ch/units/651;75;5;https://orca.mova21.ch/activities/75
651;wolf;de;7;0;https://orca.mova21.ch/units/651;237;6;https://orca.mova21.ch/activities/237
651;wolf;de;7;0;https://orca.mova21.ch/units/651;96;7;https://orca.mova21.ch/activities/96
651;wolf;de;7;0;https://orca.mova21.ch/units/651;163;8;https://orca.mova21.ch/activities/163
651;wolf;de;7;0;https://orca.mova21.ch/units/651;222;9;https://orca.mova21.ch/activities/222
651;wolf;de;7;0;https://orca.mova21.ch/units/651;220;10;https://orca.mova21.ch/activities/220
651;wolf;de;7;0;https://orca.mova21.ch/units/651;18;11;https://orca.mova21.ch/activities/18
651;wolf;de;7;0;https://orc

In [11]:
logger = Logger(filename="countOfActivities.csv")
report.getBookingCountOfActivity()
logger.flush()

act_id;#bookings;#numberOfPeopleAssigned;activity_url
156;417;7405;https://orca.mova21.ch/activities/156
30;361;7103;https://orca.mova21.ch/activities/30
237;341;5548;https://orca.mova21.ch/activities/237
169;330;5920;https://orca.mova21.ch/activities/169
107;314;6539;https://orca.mova21.ch/activities/107
222;313;6099;https://orca.mova21.ch/activities/222
223;293;5910;https://orca.mova21.ch/activities/223
42;286;5868;https://orca.mova21.ch/activities/42
17;272;5470;https://orca.mova21.ch/activities/17
178;253;4622;https://orca.mova21.ch/activities/178
122;243;5376;https://orca.mova21.ch/activities/122
173;238;4954;https://orca.mova21.ch/activities/173
172;219;4195;https://orca.mova21.ch/activities/172
146;213;3666;https://orca.mova21.ch/activities/146
176;208;4013;https://orca.mova21.ch/activities/176
126;205;4377;https://orca.mova21.ch/activities/126
16;205;4182;https://orca.mova21.ch/activities/16
20;205;4139;https://orca.mova21.ch/activities/20
187;199;3921;https://orca.mova21.ch/ac

In [12]:
logger = Logger(filename="countOfActivitiesWeighted.csv")
report.getBookingCountOfActivityWeighted()
logger.flush()

act_id;#bookings;activity_url
156;234.30073625347472;https://orca.mova21.ch/activities/156
30;206.63962306818647;https://orca.mova21.ch/activities/30
169;178.0026740929579;https://orca.mova21.ch/activities/169
237;169.24506735159997;https://orca.mova21.ch/activities/237
17;163.37791658744794;https://orca.mova21.ch/activities/17
222;162.73814818233708;https://orca.mova21.ch/activities/222
107;154.96161429998656;https://orca.mova21.ch/activities/107
42;151.0447837909753;https://orca.mova21.ch/activities/42
223;136.86499961469622;https://orca.mova21.ch/activities/223
178;135.06446132509157;https://orca.mova21.ch/activities/178
172;118.48834667798093;https://orca.mova21.ch/activities/172
122;116.84027936776425;https://orca.mova21.ch/activities/122
173;116.68792841255944;https://orca.mova21.ch/activities/173
59;111.74402510394869;https://orca.mova21.ch/activities/59
72;110.25441218052858;https://orca.mova21.ch/activities/72
16;108.13875924360573;https://orca.mova21.ch/activities/16
176;106.